In [1]:
import math
import scipy
import numpy


def black_scholes(spot_price, strike_price, risk_rate, mature_time, volitility, callput):
    divisor = volitility * math.sqrt(mature_time)
    d1 = (math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / (divisor)
    d2 = (d1 - divisor)
    call = (spot_price * scipy.stats.norm.cdf(d1)) - strike_price * math.exp(
        -risk_rate * mature_time) * scipy.stats.norm.cdf(d2)
    put = strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(
        -d2) - spot_price * scipy.stats.norm.cdf(-d1)
    if callput == "call":
        return call
    else:
        return put
    # call = (spot_price * scipy.stats.norm.cdf((math.log(spot_price/strike_price)+(risk_rate+((volitility**2)/2))*mature_time) / (volitility*math.sqrt(mature_time)))) - strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(((math.log(spot_price/strike_price)+(risk_rate+((volitility**2)/2))*mature_time) / (volitility*math.sqrt(mature_time)) - (volitility * math.sqrt(mature_time))))


print(black_scholes(105, 120, 0.04, 8 / 12, 0.15, "call"))
print(black_scholes(105, 120, 0.04, 8 / 12, 0.15, "put"))
# print(black_scholes(100, 96, 0.05, 0.5, 0.2, "call"))
# print(black_scholes(100, 96, 0.05, 0.5, 0.2, "put"))

1.4307028251382405
13.272992747515644


In [27]:
import math

import numpy


def binomial_tree(stock_price, strike_price, interest_rate, volatility, time, steps, option_type, exercise_type):
    stock_tree = [[stock_price]]
    for step in range(1, steps + 1):
        price_row = []
        for item in range(0, step + 1):
            if item == step:
                price = stock_tree[step - 1][step - 1] * (math.exp(volatility * numpy.sqrt((time / steps))))
            elif item == 0:
                price = stock_tree[step - 1][0] * (1 / (math.exp(volatility * numpy.sqrt((time / steps)))))
            else:
                price = stock_tree[step - 1][item - 1] * (math.exp(volatility * numpy.sqrt((time / steps))))
            price_row.append(price)
        stock_tree.append(price_row)
    tree = [[-1]]
    for step in range(1, steps):
        child = []
        for item in range(step + 1):
            child.append(-1)
        tree.append(child)
    level = []
    for step in range(steps + 1):
        price = stock_tree[steps][step]
        if option_type == 'call':
            if price <= strike_price:
                value = 0
            else:
                value = price - strike_price
        else:
            if strike_price > price:
                value = strike_price - price
            else:
                value = 0
        level.append(value)
    tree.append(level)
    for i in range(steps - 1, -1, -1):
        prev_row = tree[i + 1]
        for j in range(i + 1):
            if exercise_type == 'american' and option_type == 'put':
                american_value = max(value, strike_price - stock_tree[i][j])
                item_value = (math.exp(-interest_rate * (time / steps))) * ((math.e ** (
                            interest_rate * (time / steps)) - (1 / math.e ** (
                            volatility * math.sqrt((time / steps))))) / ((math.e ** (
                            volatility * math.sqrt((time / steps)))) - (1 / math.e ** (
                            volatility * math.sqrt((time / steps))))) * prev_row[j + 1] + (1 - (
                            math.exp(interest_rate * (time / steps)) - (
                                1 / math.exp(volatility * math.sqrt((time / steps))))) / ((math.exp(
                    volatility * numpy.sqrt((time / steps)))) - (1 / math.e ** (
                            volatility * numpy.sqrt((time / steps)))))) * prev_row[j])
                tree[i][j] = max(american_value, item_value)
            else:
                tree[i][j] = (math.exp(-interest_rate * (time / steps))) * ((math.e ** (
                            interest_rate * (time / steps)) - (1 / math.e ** (
                            volatility * math.sqrt((time / steps))))) / ((math.e ** (
                            volatility * math.sqrt((time / steps)))) - (1 / math.e ** (
                            volatility * math.sqrt((time / steps))))) * prev_row[j + 1] + (1 - (
                            math.e ** (interest_rate * (time / steps)) - (
                                1 / math.e ** (volatility * math.sqrt((time / steps))))) / ((math.e ** (
                            volatility * math.sqrt((time / steps)))) - (1 / math.e ** (
                            volatility * math.sqrt((time / steps)))))) * prev_row[j])
    tree = tree[0][0]
    return numpy.round(tree, decimals=1)


print(binomial_tree(100, 120, 0.05, 0.20, 0.5, 180, 'call', 'europiean'))
print(binomial_tree(100, 120, 0.05, 0.20, 0.5, 180, 'put', 'europiean'))
print(binomial_tree(100, 120, 0.05, 0.20, 0.5, 180, 'call', 'american'))
print(binomial_tree(100, 120, 0.05, 0.20, 0.5, 180, 'put', 'american'))
# print('--')
print(binomial_tree(100, 98, 0.05, 0.20, 0.5, 180, 'call', 'europiean'))
print(binomial_tree(100, 98, 0.05, 0.20, 0.5, 180, 'put', 'europiean'))
print(binomial_tree(100, 98, 0.05, 0.20, 0.5, 180, 'call', 'american'))
print(binomial_tree(100, 98, 0.05, 0.20, 0.5, 180, 'put', 'american'))


1.0
18.1
1.0
20
10.5
2.2
10.5
2.3


In [3]:
import scipy


def monte_carlo(stock_price, strike_price, time, interest_rate, volatility, type, steps, simulations):
    st = stock_price * numpy.exp(numpy.cumsum(((interest_rate - 0.5 * volatility * volatility) * time / steps) + (
            volatility * numpy.sqrt(time / steps)) * numpy.random.standard_normal(size=(steps, simulations)), axis=0))
    if type == "put":
        values = numpy.maximum(strike_price - st[-1], 0)
    else:
        values = numpy.maximum(st[-1] - strike_price, 0)
    val = 0
    for i in range(0, len(values)):
        val = values[i] + val
    return numpy.round(val / len(values) * math.exp(-interest_rate * time), decimals=1)


print(monte_carlo(100, 120, 0.5, 0.05, 0.2, "call", 500, 200000))
print(monte_carlo(100, 120, 0.5, 0.05, 0.2, "put", 500, 200000))
print(monte_carlo(100, 96, 0.5, 0.05, 0.2, "call", 500, 200000))
print(monte_carlo(100, 96, 0.5, 0.05, 0.2, "put", 500, 200000))

1.0
18.0
9.2
2.9
